# Лабораторная работа №4. Использование нейронных сетей для генерации текста
# Козлов, Ярикова
# Вариант №9

### Код для быстрой записи данных в таблицы (к лабе не относится)

In [1]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

class Table():
    def __init__(self, title=None, data=None, headlines=["Precision", "Recall", "F1-score", "Accuracy"],
                 indexes=["Стемминг отсутствует", "Стемминг есть",
                          "Стоп-слова есть", "Стоп-слова отсечены",
                       "Взвешивание Count", "Взвешивание TF", "Взвешивание TF-IDF",
                       "max_features 100", "max_features 1000", "max_features 2000", "max_features 5000", "max_features 10000"],
                weighted=True):
        
        self.headlines = headlines
        self.dataFrame = pd.DataFrame(columns=headlines, index=indexes)
        if weighted:
            self.dataFrame.columns = pd.MultiIndex.from_tuples(
                tuple(zip(('Weighted Avg', 'Weighted Avg', 'Weighted Avg', 'Weighted Avg'), self.dataFrame.columns)))
        
    def fill_row(self, row, data=[], pred='', target='', roc_auc=None):
        """
        Заполняет строки исходя из дефолта
        row - row index
        pred='' и target='' - полученные классификатором значения и целевые значения
        Заполняется значениями Weighted Avg"""
        if data == []:
            data.append(round(precision_score(target, pred, average='weighted'), 3))
            data.append(round(recall_score(target, pred, average='weighted'), 3))
            data.append(round(f1_score(target, pred, average='weighted'), 3))
            data.append(round(accuracy_score(target, pred,), 3))
        for i in range(len(self.headlines)):
            self.dataFrame.loc[row][i] = data[i]   
        return self.dataFrame.loc[row]
            
    def table(self):
        return self.dataFrame
        
    def __repr__(self):
        return f'{self.dataFrame}'

delimiter = "+++++++++++++++++++++++++++++++++++++++++++"
result_table = Table(indexes=["Случайный лес с оптимальными параметрами", "Метод опорных векторов с оптимальными параметрами",
                             "Случайный лес с оптимальными параметрами(Glove)", "Метод опорных векторов с оптимальными параметрами(Glove)"])

In [2]:
import tensorflow as tf
from sklearn.model_selection  import train_test_split
import numpy as np
import pandas as pd
import os
import time

In [3]:
device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

Found GPU at: 


In [4]:
RANDOM_STATE = 42

# 1. Загрузить выборку стихотворений одного из поэтов в соответствии с вариантом.

In [5]:
# Выбираем поэта
poet = 'mayakovskiy' #@param ['mayakovskiy', 'pushkin']

path_to_file = f'{poet}.txt'
path_to_file = tf.keras.utils.get_file(path_to_file, f'http://uit.mpei.ru/git/main/TDA/raw/branch/master/assets/poems/{path_to_file}')

In [6]:
# Загружаем текст из файла.
# Стихотворения в файле разделены токеном '</s>' - сохраняем в переменную
with open(path_to_file,encoding = "utf-8") as f:
    text = f.read()

print(f'Length of text: {len(text)} characters')

EOS_TOKEN = '</s>'

Length of text: 815675 characters


In [7]:
path_to_file

'C:\\Users\\Igorexy\\.keras\\datasets\\mayakovskiy.txt'

In [8]:
# Посмотрим на текст
print(text[:500])


Угрюмый дождь скосил глаза.
А за
решеткой
четкой
железной мысли проводов —
перина.
И на
нее
встающих звезд
легко оперлись ноги.
Но ги —
бель фонарей,
царей
в короне газа,
для глаза
сделала больней
враждующий букет бульварных проституток.
И жуток
шуток
клюющий смех —
из желтых
ядовитых роз
возрос
зигзагом.
За гам
и жуть
взглянуть
отрадно глазу:
раба
крестов
страдающе-спокойно-безразличных,
гроба
домов
публичных
восток бросал в одну пылающую вазу.

</s>


У —
лица.
Лица
у
догов
годов
рез —
че.
Че


# 2. Познакомиться с данными. Проанализировать статистические характеристики исходных данных (среднюю длину стихотворения, среднюю длину строки).

In [9]:
def mean_line_len(poem):
    lines = [len(line.strip()) for line in poem.split('\n') if len(line.strip())>0]
    return sum(lines)/len(lines)


def describe_poems(text,return_df = False):
    poems_list = [poem.strip() for poem in text.split(EOS_TOKEN) if len(poem.strip())>0]
    df = pd.DataFrame(data=poems_list,columns=['poem'])
    df['len'] = df.poem.map(len)
    df['lines'] = df.poem.str.count('\n')
    df['mean_line_len'] = df.poem.map(mean_line_len)
    if return_df:
        return df
    return df.describe()

In [10]:
poem_df = describe_poems(text,return_df = True)
poem_df

,poem,len,lines,mean_line_len
0,Угрюмый дождь скосил глаза.\nА за\nрешеткой\nч...,449,34,11.857143
1,У —\nлица.\nЛица\nу\nдогов\nгодов\nрез —\nче.\...,546,42,11.720930
2,"«Какая очаровательная ночь!»\n«Эта,\n(указывае...",333,21,14.181818
3,"Скрипка издергалась, упрашивая,\nи вдруг разре...",765,45,15.652174
4,Войне ли думать:\n«Некрасиво в шраме»?\nЕй ли ...,901,69,14.344828
...,...,...,...,...
738,Зеленые листики —\nи нет зимы.\nИдем\nраздолье...,279,24,10.200000
739,"У меня растут года,\nбудет и семнадцать.\nГде ...",3656,280,12.836502
740,"За море синеволное,\nза сто земель\nи вод\nраз...",666,49,12.340000
741,Уважаемые\nтоварищи потомки!\nРоясь\nв сегодня...,3681,243,14.090164


In [11]:
poem_df.describe()

,len,lines,mean_line_len
count,743.000000,743.000000,743.000000
mean,1088.702557,81.532974,12.900151
std,623.199169,49.312455,2.385287
min,203.000000,19.000000,7.636364
25%,632.000000,44.000000,11.135068
50%,952.000000,72.000000,12.339286
75%,1345.500000,106.000000,14.682460
max,4172.000000,287.000000,18.454545


# 3. Подготовить выборку для обучения.

In [12]:
train_poems, test_poems = train_test_split(poem_df.poem.to_list(),test_size = 0.1,random_state = RANDOM_STATE)
train_poems, val_poems = train_test_split(train_poems,test_size = 0.1,random_state = RANDOM_STATE)

train_poems = f'\n\n{EOS_TOKEN}\n\n'.join(train_poems)
val_poems = f'\n\n{EOS_TOKEN}\n\n'.join(val_poems)
test_poems = f'\n\n{EOS_TOKEN}\n\n'.join(test_poems)

In [13]:
vocab = sorted(set(text))+[EOS_TOKEN]
print(f'{len(vocab)} unique characters')
print (vocab)

140 unique characters
['\n', ' ', '!', '"', '%', '&', '(', ')', ',', '-', '.', '/', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'G', 'H', 'I', 'J', 'K', 'M', 'N', 'O', 'P', 'R', 'S', 'U', 'V', 'X', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'z', '\xa0', '«', '»', 'à', 'ç', 'è', 'ö', 'ü', '̀', '́', '·', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '–', '—', '’', '…', '№', '</s>']


In [14]:
# Для подачи на вход нейронной сети необходимо закодировать текст в виде числовой последовательности.
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1).numpy().decode('utf-8')
    
def ids_from_text(text):
    return ids_from_chars(tf.strings.unicode_split(text, input_encoding='UTF-8'))

In [15]:
# пример кодирования
ids = ids_from_text(train_poems[:20])
res_text = text_from_ids(ids)
print(train_poems[:20],ids,res_text,sep = '\n')

У Петровой
у Надежды
tf.Tensor(
[ 90   2  86 107 120 118 116 104 116 111   1 121   2  84 102 106 107 108
 106 129], shape=(20,), dtype=int64)
У Петровой
у Надежды


In [16]:
# Кодируем данные и преобразуем их в Датасеты
train_ids = ids_from_text(train_poems)
val_ids = ids_from_text(val_poems)
test_ids = ids_from_text(test_poems)

train_ids_dataset = tf.data.Dataset.from_tensor_slices(train_ids)
val_ids_dataset = tf.data.Dataset.from_tensor_slices(val_ids)
test_ids_dataset = tf.data.Dataset.from_tensor_slices(test_ids)

In [17]:
# Весь текст разбивается на последовательности длины seq_length. По этим последовательностям будет предсказываться следующий символ.
# Попробовать разные длины - среднюю длину строки, среднюю длину стиха
seq_length = 100
examples_per_epoch = len(train_ids_dataset)//(seq_length+1)

In [18]:
train_sequences = train_ids_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_ids_dataset.batch(seq_length+1, drop_remainder=True)
test_sequences = test_ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in train_sequences.take(1):
  print(text_from_ids(seq))

У Петровой
у Надежды
не имеется одежды.
Чтоб купить
(пришли деньки!),
не имеется деньги́.
Ей
в расцве


In [19]:
# Создаем датасет с input и target строками
# target сдвинута относительно input на один символ.
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
train_dataset = train_sequences.map(split_input_target)
val_dataset = val_sequences.map(split_input_target)
test_dataset = test_sequences.map(split_input_target)

In [21]:
for input_example, target_example in val_dataset.take(1):
    print("Input :", text_from_ids(input_example))
    print("Target:", text_from_ids(target_example))

Input : В смокинг вштопорен,
побрит что надо.
По гранд
по опере
гуляю грандом.
Смотрю
в антракте —
красавка 
Target:  смокинг вштопорен,
побрит что надо.
По гранд
по опере
гуляю грандом.
Смотрю
в антракте —
красавка н


In [22]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

def prepare_dataset(dataset):
    dataset = (
        dataset
        .shuffle(BUFFER_SIZE)
        .batch(BATCH_SIZE, drop_remainder=True)
        .prefetch(tf.data.experimental.AUTOTUNE))
    return dataset 

train_dataset = prepare_dataset(train_dataset)
val_dataset = prepare_dataset(val_dataset)
test_dataset = prepare_dataset(test_dataset)

train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

# 4. Построить нейронную сеть. Тип ячейки RNN выбрать в соответствии с вариантом.

Модель состоит из трех слоев

* `tf.keras.layers.Embedding:` Входной слой. Кодирует каждый идентификатор символа в вектор размерностью embedding_dim;
* `tf.keras.layers.SimpleRNN`: Рекуррентный слой на ячейках SimpleRNN в количестве `units=rnn_units` **(Здесь нужно указать тип ячеек в соответствии с вариантом)**
* `tf.keras.layers.Dense:` Выходной полносвязный слой размерностью vocab_size, в который выводится вероятность каждого символа в словаре.

In [23]:
# Длина словаря символов
vocab_size = len(vocab)

# размерность Embedding'а
embedding_dim = 256 #@param{type:"number"}

# Параметры RNN-слоя
rnn_units = 300 #@param {type:"number"}
dropout_p = 0.5

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.simple = tf.keras.layers.SimpleRNN(rnn_units,
                                   dropout = dropout_p,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    
    if states is None:
        states = self.simple.get_initial_state(x)

    x, *states = self.simple(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [25]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### Проверка необученой модели

In [26]:
# посмотрим на один батч из датасета
for input_example_batch, target_example_batch in train_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 141) # (batch_size, sequence_length, vocab_size)


In [27]:
example_batch_predictions[0][0]

<tf.Tensor: shape=(141,), dtype=float32, numpy=
array([ 0.04580479,  0.02026156, -0.02932041,  0.01080876,  0.03008067,
       -0.01397174, -0.04088385,  0.00980235, -0.01136458, -0.03861335,
        0.03159821, -0.03741074,  0.00188887, -0.03991881, -0.05089642,
       -0.00251933,  0.04797433,  0.00983226,  0.03675636, -0.00229334,
        0.01785504,  0.02266506, -0.04054524, -0.00098924,  0.01531633,
        0.04144016, -0.01244012,  0.00970926,  0.02226621,  0.05057633,
       -0.02856871, -0.03057659, -0.05018119,  0.01293335, -0.01567086,
        0.02911002,  0.02528487,  0.01414949,  0.00015195, -0.06126463,
       -0.02632061,  0.02134457,  0.03885552,  0.01414965, -0.00763898,
       -0.00962067,  0.01888105,  0.01272053,  0.00853734,  0.01946685,
        0.02413196,  0.01342708,  0.03141851,  0.02057206, -0.02138245,
       -0.02276708,  0.01623563,  0.0356742 ,  0.04675717, -0.00521255,
        0.00065888, -0.02123081, -0.01535884, -0.01793941,  0.0008652 ,
        0.019045

In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([113, 113,  67,  22,   4, 133,  68,   2,  16, 138,   4,  49,  99,
        86,  49,  17, 138,  10, 136,  68,  28,  27, 132,  41,  84, 102,
        54,  72,  66,  30,  98,  58,  94,  34,  86,  98, 105, 140,  95,
       114,  24, 124,  20,  73, 134,  95,  37,  85,  46,  91, 133, 112,
        22,  93, 138, 106, 136,  70,  44, 133,  47, 139,  84, 124, 116,
       108, 130,  80,  78,  50,  32,  10,  61,  49,  83,  58,  49,   9,
        86, 124,  46,  38,  53, 135, 114,  97,   9,  49,  86, 137,  29,
        45, 138,  70, 101, 105, 131, 127,  23,  26], dtype=int64)

In [29]:
print("Input:\n", text_from_ids(input_example_batch[0]))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices))

Input:
 трочки,
краснодеревщики
не слали мебель на́ дом.
И кроме
свежевымытой сорочки,
скажу по совести,
мне

Next Char Predictions:
 ллüE"я̀ >…"mЭПm?…-—̀MKюdНаsБöOЬwЧUПЬг</s>ШмHцCВёШZОiФякEЦ…д—·gяj№НцожьЙЗnR-«mМwm,Пцiar–мЫ,mП’Nh…·ЯгэщGJ


# 5. Обучить нейронную сеть на разных количествах эпох (5, 15, 30, 50, 70) при зафиксированных параметрах embedding_dim = 256, rnn_units = 300, T = 0.3 и сравнить результаты генерации (тексты), перплексию и статистические характеристики сгенерированных текстов. Выбрать оптимальное количество эпох

### Обучение модели

Можно представить задачу как задачу классификации - по предыдущему состоянию RNN и входу в данный момент времени предсказать класс (очередной символ).
В этом случае работает стандартная функция потерь tf.keras.losses.sparse_categorical_crossentropy- кроссэнтропия, которая равна минус логарифму предсказанной вероятности для верного класса.

Поскольку модель возвращает логиты, вам необходимо установить флаг from_logits.

In [30]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 141)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.9636793, shape=(), dtype=float32)


Необученная модель не может делать адекватные предсказания. Ее перплексия («коэффициент неопределённости») приблизительно равна размеру словаря. Это говорит о полной неопределенности модели при генерации текста.

Перплексия = exp(кроссэнтропия)

In [32]:
print('perplexity: ',np.exp(example_batch_mean_loss))

perplexity:  143.11942


Настраиваем обучение, используя метод tf.keras.Model.compile. Используйте tf.keras.optimizers.Adam с аргументами по умолчанию и функцией потерь.

In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  36096     
                                                                 
 simple_rnn (SimpleRNN)      multiple                  167100    
                                                                 
 dense (Dense)               multiple                  42441     
                                                                 
Total params: 245,637
Trainable params: 245,637
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Используем tf.keras.callbacks.ModelCheckpoint, чтобы убедиться, что контрольные точки сохраняются во время обучения:
# Directory where the checkpoints will be saved

checkpoint_dir = './training_checkpoints'
if not os.path.exists(checkpoint_dir):
    # Если папки нет, создаем ее
    os.makedirs(checkpoint_dir)

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor="val_loss",
    save_weights_only=True,
    save_best_only=True)

# Генерация текста

In [36]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    
  # Этот фрагмент целиком написан с использованием Tensorflow, поэтому его можно выполнять 
  # не с помощью интерпретатора языка Python, а через граф операций. Это будет значительно быстрее.  
  # Для этого воспользуемся декоратором  @tf.function   
  @tf.function   
  def generate_one_step(self, inputs, states=None,temperature=1.0):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)


    # Return the characters and model state.
    return predicted_chars, states

In [37]:
result_table_EPOCHS = Table(headlines=['eval loss', 'perplexity', 'result_text', 'Run time'], 
                     indexes=['EPOCHS 5', 'EPOCHS 15', 'EPOCHS 30', 'EPOCHS 50', 'EPOCHS 70'], weighted=False)
describe_poems_dict_EPOCHS = {}

In [38]:
# Длина словаря символов
vocab_size = len(vocab)

# размерность Embedding'а
embedding_dim = 256 #@param{type:"number"}

# Параметры RNN-слоя
rnn_units = 300 #@param {type:"number"}
dropout_p = 0.5

T = 0.3 #@param {type:"slider", min:0, max:2, step:0.1}
N = 1000

EPOCHS = [5, 15, 30, 50, 70]

In [39]:
for i in EPOCHS:
    history = model.fit(train_dataset, validation_data = val_dataset, epochs=i, callbacks=[checkpoint_callback])
    eval_loss = model.evaluate(test_dataset)
    perplexity = np.exp(eval_loss)
    print('eval loss:', eval_loss)
    print('perplexity', np.exp(eval_loss))
    
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
    
    start = time.time()
    states = None
    next_char = tf.constant(['\n'])
    result = [next_char]

    for n in range(N):
        next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()

    result_text = result[0].numpy().decode('utf-8')
    print(result_text)
    print('_'*80)
    Run_time = end - start
    print('\nRun time:', Run_time)
    
    result_table_EPOCHS.fill_row(row='EPOCHS '+str(i), data=[eval_loss, perplexity, result_text, Run_time])
    describe_poems_dict_EPOCHS.update({i: describe_poems(result_text)})

Epoch 1/5
102/102 [==============================] - 12s 105ms/step - loss: 3.2137 - val_loss: 2.7415
Epoch 2/5
102/102 [==============================] - 10s 91ms/step - loss: 2.6613 - val_loss: 2.5758
Epoch 3/5
102/102 [==============================] - 10s 95ms/step - loss: 2.5582 - val_loss: 2.4985
Epoch 4/5
102/102 [==============================] - 10s 91ms/step - loss: 2.4970 - val_loss: 2.4497
Epoch 5/5
12/12 [==============================] - 0s 34ms/step - loss: 2.4053
eval loss: 2.405268430709839
perplexity 11.08140447225509

Всем бростая.
В странной в долонки.

</s>

Верет сталить на поровать страте —
и сторований востанной
стротит
стором породите.
Пот бороните на поровите,
как породить волонной лизит
в отонь,
половой поредеть —
молонить,
стоть
в стопуть в простома —
не на постару.
В сторот в востовать —
не строть не водет
посторами серется
в разной поредита.
В сторать в кольше,
серет породное старища.
Разовеникой морок.
Собот каренный
как не простовать не настью —
велит
и 

102/102 [==============================] - 9s 85ms/step - loss: 2.0135 - val_loss: 2.0276
Epoch 2/50
102/102 [==============================] - 9s 86ms/step - loss: 2.0129 - val_loss: 2.0265
Epoch 3/50
102/102 [==============================] - 9s 86ms/step - loss: 2.0091 - val_loss: 2.0248
Epoch 4/50
102/102 [==============================] - 9s 88ms/step - loss: 2.0074 - val_loss: 2.0229
Epoch 5/50
102/102 [==============================] - 9s 87ms/step - loss: 2.0057 - val_loss: 2.0255
Epoch 6/50
102/102 [==============================] - 9s 86ms/step - loss: 2.0046 - val_loss: 2.0218
Epoch 7/50
102/102 [==============================] - 9s 87ms/step - loss: 2.0007 - val_loss: 2.0196
Epoch 8/50
102/102 [==============================] - 9s 87ms/step - loss: 1.9991 - val_loss: 2.0188
Epoch 9/50
102/102 [==============================] - 9s 85ms/step - loss: 1.9973 - val_loss: 2.0211
Epoch 10/50
102/102 [==============================] - 9s 88ms/step - loss: 1.9957 - val_loss: 2.0178


Epoch 19/70
102/102 [==============================] - 10s 89ms/step - loss: 1.9287 - val_loss: 1.9871
Epoch 20/70
102/102 [==============================] - 10s 89ms/step - loss: 1.9266 - val_loss: 1.9854
Epoch 21/70
102/102 [==============================] - 10s 90ms/step - loss: 1.9258 - val_loss: 1.9843
Epoch 22/70
102/102 [==============================] - 10s 92ms/step - loss: 1.9264 - val_loss: 1.9838
Epoch 23/70
102/102 [==============================] - 10s 90ms/step - loss: 1.9247 - val_loss: 1.9845
Epoch 24/70
102/102 [==============================] - 10s 90ms/step - loss: 1.9243 - val_loss: 1.9833
Epoch 25/70
102/102 [==============================] - 10s 93ms/step - loss: 1.9248 - val_loss: 1.9835
Epoch 26/70
102/102 [==============================] - 10s 97ms/step - loss: 1.9228 - val_loss: 1.9833
Epoch 27/70
102/102 [==============================] - 9s 88ms/step - loss: 1.9226 - val_loss: 1.9854
Epoch 28/70
102/102 [==============================] - 10s 90ms/step - los

# 6. Изменяя параметр температуры T проанализировать изменения сгенерированного текста. Выбрать оптимальное значение параметра.

In [40]:
Temperature = [0.1, 0.3, 0.5, 0.7, 1.0]

In [41]:
for temp in Temperature:
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

    start = time.time()
    states = None
    next_char = tf.constant(['\n'])
    result = [next_char]

    for n in range(N):
        next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=temp)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()

    result_text = result[0].numpy().decode('utf-8')
    print('_'*80, f"T={temp}", sep='\n')
    print(result_text)
    print('_'*80)
    Run_time = end - start
    print('\nRun time:', Run_time)
    print('\n')

________________________________________________________________________________
T=0.1

и стальные день
в странах
не стальные день
в странах волны,
в странах старого —
в странах крестьянский —
и в странах под красной странах
в странах страна —
не стальные странах
в положить в старой —
в странах страна —
в странах
на странах
в странах крестьянин,
в странах волосами старается —
не стальные дома,
в странах
в странах страна —
не стальные дома,
на странах
не стальные день —
не стальные волны,
в странах страна —
под красной странах —
не стальные делаться.
Под красной странах —
под красной странах
в странах страна —
не стали по старом странах —
не стальные день
в странах
стальные странах —
не стальные странах
страна —
не стал и старая воздух,
на странах старая в старой —
стальные странах —
в странах
в странах страна —
в странах казане.

</s>

В каждой страна —
не стали в коммунистых не старая,
в странах страна —
и в старом странах —
в странах страна вода.
В каждый странах
в странах дело —
не 

Сложно визуально определить лучшее значение T

# 7. Проанализировать зависимость перплексии, скорости обучения, результатов генерации от параметров нейронной сети embedding_dim и rnn_units:
* `embedding_dim` = {vocab/4, vocab/2, vocab, vocab * 2, vocab * 4}, где vocab = размер словаря выборки.
* `rnn_units` = {10, 100, 300, 500}

In [42]:
result_table_embedding_dim = Table(headlines=['eval loss', 'perplexity', 'result_text', 'Run time'], 
                     indexes=['embedding_dim 35', 'embedding_dim 70', 'embedding_dim 140', 'embedding_dim 280', 'embedding_dim 560'], weighted=False)
describe_poems_dict_embedding_dim = {}

In [56]:
# Длина словаря символов
vocab_size = len(vocab)

# размерность Embedding'а
embedding_dim = [len(vocab)/4, len(vocab)/2, len(vocab), len(vocab) * 2, len(vocab) * 4] #@param{type:"number"}
embedding_dim = list(map(int, embedding_dim))

# Параметры RNN-слоя
rnn_units = 300 #@param {type:"number"}
dropout_p = 0.5

T = 0.3 #@param {type:"slider", min:0, max:2, step:0.1}
N = 1000

In [59]:
for embedding in embedding_dim:
    model = MyModel(
        vocab_size=len(ids_from_chars.get_vocabulary()),
        embedding_dim=embedding,
        rnn_units=rnn_units)
    model.compile(optimizer='adam', loss=loss)

    history = model.fit(train_dataset, validation_data = val_dataset, epochs=70, callbacks=[checkpoint_callback])
    eval_loss = model.evaluate(test_dataset)
    perplexity = np.exp(eval_loss)
    print('eval loss:', eval_loss)
    print('perplexity', np.exp(eval_loss))

    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

    start = time.time()
    states = None
    next_char = tf.constant(['\n'])
    result = [next_char]

    for n in range(N):
        next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()

    result_text = result[0].numpy().decode('utf-8')
    print(result_text)
    print('_'*80)
    Run_time = end - start
    print('\nRun time:', Run_time)

    result_table_embedding_dim.fill_row(row='embedding_dim ' + str(int(embedding)), data=[eval_loss, perplexity, result_text, Run_time])
    describe_poems_dict_embedding_dim.update({int(embedding): describe_poems(result_text)})

Epoch 1/70
102/102 [==============================] - 8s 69ms/step - loss: 3.4805 - val_loss: 3.1441
Epoch 2/70
102/102 [==============================] - 9s 80ms/step - loss: 2.9587 - val_loss: 2.9159
Epoch 3/70
102/102 [==============================] - 9s 82ms/step - loss: 2.8400 - val_loss: 2.7388
Epoch 4/70
102/102 [==============================] - 9s 83ms/step - loss: 2.7585 - val_loss: 2.6824
Epoch 5/70
102/102 [==============================] - 9s 84ms/step - loss: 2.7530 - val_loss: 2.6629
Epoch 6/70
102/102 [==============================] - 9s 81ms/step - loss: 2.6954 - val_loss: 2.6266
Epoch 7/70
102/102 [==============================] - 10s 90ms/step - loss: 2.6675 - val_loss: 2.6009
Epoch 8/70
102/102 [==============================] - 9s 82ms/step - loss: 2.6448 - val_loss: 2.5808
Epoch 9/70
102/102 [==============================] - 8s 75ms/step - loss: 2.6246 - val_loss: 2.5613
Epoch 10/70
102/102 [==============================] - 9s 79ms/step - loss: 2.6031 - val_l

102/102 [==============================] - 9s 76ms/step - loss: 3.4365 - val_loss: 2.9904
Epoch 2/70
102/102 [==============================] - 8s 78ms/step - loss: 2.8285 - val_loss: 2.7015
Epoch 3/70
102/102 [==============================] - 9s 83ms/step - loss: 2.6917 - val_loss: 2.6193
Epoch 4/70
102/102 [==============================] - 8s 74ms/step - loss: 2.6293 - val_loss: 2.5686
Epoch 5/70
102/102 [==============================] - 8s 77ms/step - loss: 2.5845 - val_loss: 2.5213
Epoch 6/70
102/102 [==============================] - 8s 72ms/step - loss: 2.5457 - val_loss: 2.4856
Epoch 7/70
102/102 [==============================] - 8s 75ms/step - loss: 2.5164 - val_loss: 2.4597
Epoch 8/70
102/102 [==============================] - 8s 76ms/step - loss: 2.4904 - val_loss: 2.4351
Epoch 9/70
102/102 [==============================] - 9s 80ms/step - loss: 2.4658 - val_loss: 2.4106
Epoch 10/70
102/102 [==============================] - 9s 78ms/step - loss: 2.4465 - val_loss: 2.3883


102/102 [==============================] - 16s 135ms/step - loss: 3.3181 - val_loss: 2.8290
Epoch 2/70
102/102 [==============================] - 13s 118ms/step - loss: 2.7300 - val_loss: 2.6345
Epoch 3/70
102/102 [==============================] - 13s 119ms/step - loss: 2.6186 - val_loss: 2.5536
Epoch 4/70
102/102 [==============================] - 13s 116ms/step - loss: 2.5570 - val_loss: 2.5047
Epoch 5/70
102/102 [==============================] - 14s 131ms/step - loss: 2.5116 - val_loss: 2.4646
Epoch 6/70
102/102 [==============================] - 14s 129ms/step - loss: 2.4768 - val_loss: 2.4299
Epoch 7/70
102/102 [==============================] - 14s 128ms/step - loss: 2.4462 - val_loss: 2.4019
Epoch 8/70
102/102 [==============================] - 15s 137ms/step - loss: 2.4189 - val_loss: 2.3808
Epoch 9/70
102/102 [==============================] - 15s 136ms/step - loss: 2.3953 - val_loss: 2.3532
Epoch 10/70
102/102 [==============================] - 15s 136ms/step - loss: 2.3742

102/102 [==============================] - 17s 142ms/step - loss: 3.1949 - val_loss: 2.7306
Epoch 2/70
102/102 [==============================] - 16s 147ms/step - loss: 2.6455 - val_loss: 2.5636
Epoch 3/70
102/102 [==============================] - 15s 139ms/step - loss: 2.5460 - val_loss: 2.4906
Epoch 4/70
102/102 [==============================] - 15s 136ms/step - loss: 2.4862 - val_loss: 2.4399
Epoch 5/70
102/102 [==============================] - 14s 128ms/step - loss: 2.4419 - val_loss: 2.4026
Epoch 6/70
102/102 [==============================] - 15s 136ms/step - loss: 2.4057 - val_loss: 2.3690
Epoch 7/70
102/102 [==============================] - 17s 155ms/step - loss: 2.3741 - val_loss: 2.3413
Epoch 8/70
102/102 [==============================] - 17s 154ms/step - loss: 2.3464 - val_loss: 2.3144
Epoch 9/70
102/102 [==============================] - 17s 159ms/step - loss: 2.3205 - val_loss: 2.2887
Epoch 10/70
102/102 [==============================] - 17s 157ms/step - loss: 2.2966

102/102 [==============================] - 28s 240ms/step - loss: 3.0948 - val_loss: 2.6477
Epoch 2/70
102/102 [==============================] - 25s 235ms/step - loss: 2.5825 - val_loss: 2.5113
Epoch 3/70
102/102 [==============================] - 25s 237ms/step - loss: 2.4884 - val_loss: 2.4447
Epoch 4/70
102/102 [==============================] - 26s 245ms/step - loss: 2.4303 - val_loss: 2.3946
Epoch 5/70
102/102 [==============================] - 24s 229ms/step - loss: 2.3845 - val_loss: 2.3514
Epoch 6/70
102/102 [==============================] - 26s 248ms/step - loss: 2.3474 - val_loss: 2.3179
Epoch 7/70
102/102 [==============================] - 24s 228ms/step - loss: 2.3122 - val_loss: 2.2842
Epoch 8/70
102/102 [==============================] - 24s 227ms/step - loss: 2.2833 - val_loss: 2.2583
Epoch 9/70
102/102 [==============================] - 24s 228ms/step - loss: 2.2553 - val_loss: 2.2318
Epoch 10/70
102/102 [==============================] - 26s 242ms/step - loss: 2.2314

In [60]:
result_table_rnn_units = Table(headlines=['eval loss', 'perplexity', 'result_text', 'Run time'], 
                     indexes=['rnn_units 10', 'rnn_units 100', 'rnn_units 300', 'rnn_units 500'], weighted=False)
describe_poems_dict_rnn_units = {}

In [61]:
# Длина словаря символов
vocab_size = len(vocab)

# размерность Embedding'а
embedding_dim = 256 #@param{type:"number"}

# Параметры RNN-слоя
rnn_units = [10, 100, 300, 500] #@param {type:"number"}
dropout_p = 0.5

T = 0.3 #@param {type:"slider", min:0, max:2, step:0.1}
N = 1000

In [63]:
for rnn_un in rnn_units:
    model = MyModel(
        vocab_size=len(ids_from_chars.get_vocabulary()),
        embedding_dim=embedding_dim,
        rnn_units=rnn_un)
    model.compile(optimizer='adam', loss=loss)

    history = model.fit(train_dataset, validation_data = val_dataset, epochs=70, callbacks=[checkpoint_callback])
    eval_loss = model.evaluate(test_dataset)
    perplexity = np.exp(eval_loss)
    print('eval loss:', eval_loss)
    print('perplexity', np.exp(eval_loss))

    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

    start = time.time()
    states = None
    next_char = tf.constant(['\n'])
    result = [next_char]

    for n in range(N):
        next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()

    result_text = result[0].numpy().decode('utf-8')
    print(result_text)
    print('_'*80)
    Run_time = end - start
    print('\nRun time:', Run_time)

    result_table_rnn_units.fill_row(row='rnn_units ' + str(int(rnn_un)), data=[eval_loss, perplexity, result_text, Run_time])
    describe_poems_dict_rnn_units.update({int(rnn_un): describe_poems(result_text)})

Epoch 1/70
102/102 [==============================] - 5s 32ms/step - loss: 4.3768 - val_loss: 3.8075
Epoch 2/70
102/102 [==============================] - 4s 35ms/step - loss: 3.6164 - val_loss: 3.4918
Epoch 3/70
102/102 [==============================] - 4s 29ms/step - loss: 3.4330 - val_loss: 3.3889
Epoch 4/70
102/102 [==============================] - 4s 31ms/step - loss: 3.3555 - val_loss: 3.3229
Epoch 5/70
102/102 [==============================] - 4s 32ms/step - loss: 3.2799 - val_loss: 3.2442
Epoch 6/70
102/102 [==============================] - 4s 32ms/step - loss: 3.2064 - val_loss: 3.1544
Epoch 7/70
102/102 [==============================] - 4s 32ms/step - loss: 3.1018 - val_loss: 3.0600
Epoch 8/70
102/102 [==============================] - 4s 36ms/step - loss: 3.0231 - val_loss: 2.9916
Epoch 9/70
102/102 [==============================] - 5s 40ms/step - loss: 2.9638 - val_loss: 2.9374
Epoch 10/70
102/102 [==============================] - 5s 38ms/step - loss: 2.9143 - val_lo

102/102 [==============================] - 6s 45ms/step - loss: 3.4519 - val_loss: 2.9050
Epoch 2/70
102/102 [==============================] - 5s 43ms/step - loss: 2.7643 - val_loss: 2.6624
Epoch 3/70
102/102 [==============================] - 5s 44ms/step - loss: 2.6317 - val_loss: 2.5809
Epoch 4/70
102/102 [==============================] - 5s 43ms/step - loss: 2.5742 - val_loss: 2.5340
Epoch 5/70
102/102 [==============================] - 5s 44ms/step - loss: 2.5351 - val_loss: 2.5009
Epoch 6/70
102/102 [==============================] - 5s 43ms/step - loss: 2.5051 - val_loss: 2.4720
Epoch 7/70
102/102 [==============================] - 5s 41ms/step - loss: 2.4806 - val_loss: 2.4498
Epoch 8/70
102/102 [==============================] - 4s 39ms/step - loss: 2.4601 - val_loss: 2.4304
Epoch 9/70
102/102 [==============================] - 5s 41ms/step - loss: 2.4429 - val_loss: 2.4160
Epoch 10/70
102/102 [==============================] - 4s 39ms/step - loss: 2.4279 - val_loss: 2.4003


102/102 [==============================] - 16s 142ms/step - loss: 3.2083 - val_loss: 2.7397
Epoch 2/70
102/102 [==============================] - 14s 133ms/step - loss: 2.6527 - val_loss: 2.5719
Epoch 3/70
102/102 [==============================] - 15s 142ms/step - loss: 2.5547 - val_loss: 2.4995
Epoch 4/70
102/102 [==============================] - 15s 138ms/step - loss: 2.4940 - val_loss: 2.4473
Epoch 5/70
102/102 [==============================] - 14s 128ms/step - loss: 2.4510 - val_loss: 2.4078
Epoch 6/70
102/102 [==============================] - 14s 132ms/step - loss: 2.4149 - val_loss: 2.3752
Epoch 7/70
102/102 [==============================] - 14s 132ms/step - loss: 2.3842 - val_loss: 2.3478
Epoch 8/70
102/102 [==============================] - 13s 126ms/step - loss: 2.3554 - val_loss: 2.3203
Epoch 9/70
102/102 [==============================] - 14s 131ms/step - loss: 2.3298 - val_loss: 2.2968
Epoch 10/70
102/102 [==============================] - 14s 129ms/step - loss: 2.3059

102/102 [==============================] - 24s 218ms/step - loss: 3.1880 - val_loss: 2.7396
Epoch 2/70
102/102 [==============================] - 22s 215ms/step - loss: 2.6563 - val_loss: 2.5716
Epoch 3/70
102/102 [==============================] - 23s 221ms/step - loss: 2.5563 - val_loss: 2.5053
Epoch 4/70
102/102 [==============================] - 22s 215ms/step - loss: 2.4958 - val_loss: 2.4474
Epoch 5/70
102/102 [==============================] - 22s 212ms/step - loss: 2.4503 - val_loss: 2.4012
Epoch 6/70
102/102 [==============================] - 22s 213ms/step - loss: 2.4065 - val_loss: 2.3653
Epoch 7/70
102/102 [==============================] - 22s 214ms/step - loss: 2.3688 - val_loss: 2.3319
Epoch 8/70
102/102 [==============================] - 22s 215ms/step - loss: 2.3347 - val_loss: 2.2996
Epoch 9/70
102/102 [==============================] - 22s 213ms/step - loss: 2.3012 - val_loss: 2.2685
Epoch 10/70
102/102 [==============================] - 22s 213ms/step - loss: 2.2708

# Выводы:

### Таблица по сравнению эпох

In [64]:
result_table_EPOCHS.table()

,eval loss,perplexity,result_text,Run time
EPOCHS 5,2.405268,11.081404,\nВсем бростая.\nВ странной в долонки.\n\n</s>...,1.040562
EPOCHS 15,2.144165,8.534912,\nне света\nна положенный стой!\nХоть на нет н...,0.905763
EPOCHS 30,2.041177,7.699665,\nодно сказать в гром деньгу дома\nне выставил...,1.061969
EPOCHS 50,2.003134,7.412246,\n\n\n\n\n\n\n\nВот —\nи простоло\nи в разумее...,1.284416
EPOCHS 70,1.997716,7.372198,\nи в старом востока\nи в стране\nсветите\nне ...,1.361585


In [65]:
describe_poems_dict_EPOCHS

{5:               len      lines  mean_line_len
 count    2.000000   2.000000       2.000000
 mean   496.000000  28.000000      16.553571
 std    651.952452  38.183766       0.631345
 min     35.000000   1.000000      16.107143
 25%    265.500000  14.500000      16.330357
 50%    496.000000  28.000000      16.553571
 75%    726.500000  41.500000      16.776786
 max    957.000000  55.000000      17.000000,
 15:           len  lines  mean_line_len
 count     1.0    1.0           1.00
 mean   1000.0   53.0          18.94
 std       NaN    NaN            NaN
 min    1000.0   53.0          18.94
 25%    1000.0   53.0          18.94
 50%    1000.0   53.0          18.94
 75%    1000.0   53.0          18.94
 max    1000.0   53.0          18.94,
 30:               len      lines  mean_line_len
 count    3.000000   3.000000       3.000000
 mean   328.000000  14.333333      19.807217
 std    233.231645  10.692677       3.697668
 min    101.000000   5.000000      16.000000
 25%    208.500000   8.5

### Таблица по сравнению embedding_dim 

In [66]:
result_table_embedding_dim.table()

,eval loss,perplexity,result_text,Run time
embedding_dim 35,2.155928,8.635904,\nс дело\nпо стальный под под страна.\nПостопл...,1.147767
embedding_dim 70,2.087182,8.062162,\nне все в забетать в страно —\nи столет и в к...,1.668085
embedding_dim 140,2.050038,7.768195,"\nв страна\nс порох,\nпока\nне под ногам\nв но...",1.899963
embedding_dim 280,2.024001,7.568549,\nпоставить на всегда —\nв под вас\nне в стран...,1.544119
embedding_dim 560,2.00957,7.460111,"\nпо слова с париж,\nи вот такого столица.\nНа...",1.631322


In [67]:
describe_poems_dict_embedding_dim

{35:               len      lines  mean_line_len
 count    5.000000   5.000000       5.000000
 mean   193.600000   8.600000      18.291823
 std    162.653927   7.503333       1.849337
 min     33.000000   1.000000      16.000000
 25%     77.000000   3.000000      16.857143
 50%    124.000000   6.000000      18.500000
 75%    340.000000  15.000000      19.789474
 max    394.000000  18.000000      20.312500,
 70:               len      lines  mean_line_len
 count    2.000000   2.000000       2.000000
 mean   496.000000  24.000000      22.098485
 std    470.933116  26.870058       5.988980
 min    163.000000   5.000000      17.863636
 25%    329.500000  14.500000      19.981061
 50%    496.000000  24.000000      22.098485
 75%    662.500000  33.500000      24.215909
 max    829.000000  43.000000      26.333333,
 140:               len      lines  mean_line_len
 count    3.000000   3.000000       3.000000
 mean   328.000000  17.666667      16.975678
 std    230.922065  13.650397       0.92

### Таблица по сравнению rnn_units 

In [68]:
result_table_rnn_units.table()

,eval loss,perplexity,result_text,Run time
rnn_units 10,2.595231,13.39968,\nчто болать —\nна —\nвом светом позонова кали...,1.07543
rnn_units 100,2.204956,9.069851,"\nна порасты и ворость,\nна нам коротом тебя г...",1.731426
rnn_units 300,2.023229,7.562706,\nСоветский под нам\nне до полез дело —\nделам...,0.976045
rnn_units 500,1.986289,7.288436,"\nи столетья,\nпод носильный стране.\nСлово ст...",1.02193


In [69]:
describe_poems_dict_rnn_units

{10:           len  lines  mean_line_len
 count     1.0    1.0       1.000000
 mean   1000.0   28.0      33.517241
 std       NaN    NaN            NaN
 min    1000.0   28.0      33.517241
 25%    1000.0   28.0      33.517241
 50%    1000.0   28.0      33.517241
 75%    1000.0   28.0      33.517241
 max    1000.0   28.0      33.517241,
 100:          len  lines  mean_line_len
 count    1.0    1.0       1.000000
 mean   999.0   33.0      28.411765
 std      NaN    NaN            NaN
 min    999.0   33.0      28.411765
 25%    999.0   33.0      28.411765
 50%    999.0   33.0      28.411765
 75%    999.0   33.0      28.411765
 max    999.0   33.0      28.411765,
 300:               len     lines  mean_line_len
 count    2.000000   2.00000       2.000000
 mean   496.000000  27.50000      16.567308
 std    420.021428  21.92031       3.086793
 min    199.000000  12.00000      14.384615
 25%    347.500000  19.75000      15.475962
 50%    496.000000  27.50000      16.567308
 75%    644.500000 

# Общий вывод

`Пункт 6. Изменение температуры`
Визуально сложно определить, какое значение температуры лучше, в дальнейшем использовалось значение T=0.3

`Пункт 7. Изменение эмбеддинга`
*Эмбеддинг (embedding) — это процесс представления слов или фраз в виде векторов фиксированной длины.* При увеличении размерности вектора уменьшалась перплексия и улучшалось качество генерации текста, время на обучение увеличивалось.


`Пункт 7. Изменение параметра units в tf.keras.layers.SimpleRNN`
*Параметр units в слое tf.keras.layers.SimpleRNN определяет количество нейронов (или размерность скрытого состояния) в рекуррентном слое. Каждый нейрон в слое обрабатывает входные данные и генерирует скрытое состояние, которое передается на следующий шаг обработки.* При увеличении числа нейронов уменьшалась перплексия и улучшалось качество генерации текста, увеличивалось время обучения. Однако стоит понимать, что бесконечное увелечение нейронов может привести к переобучению.





*Перплексия (perplexity) — это метрика, используемая для оценки качества языковых моделей. Она показывает, насколько хорошо модель может предсказать следующее слово в тексте, и вычисляется как экспонента энтропии на текстовой корпус.*

In [71]:
df = result_table_EPOCHS.table()
df.to_excel("result_table_EPOCHS.xlsx")
df = result_table_embedding_dim.table()
df.to_excel("result_table_embedding_dim.xlsx")
df = result_table_rnn_units.table()
df.to_excel("result_table_rnn_units.xlsx")

In [ ]:
# # открытие файла для записи
# file = open('my_dict.json', 'r')
# # закрытие файла
# file.close()


# import json

# # создание словаря
# my_dict = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}

# # сохранение словаря в файл
# with open('my_dict.json', 'w') as file:
#     json.dump(my_dict, file)

    
# import json

# # чтение словаря из файла
# with open('my_dict.json', 'r') as file:
#     my_dict = json.load(file)


# # вывод словаря
# print(my_dict)
